Working file for Goup Project 

Data
-   https://www.fao.org/faostat/en/#data/QCL
-   
-   

Importing all libarys 

In [1]:
import pandas as pd
import numpy as np
import zipfile
import seaborn as sns # For plotting
import matplotlib.pyplot as plt # For showing plots
import scipy.stats as sts
import scikit_posthocs as sp
import statsmodels.stats as stm
from statsmodels.graphics.gofplots import qqplot


Import of data sets 

In [15]:
df = pd.read_csv("data/Production_Crops_Livestock_E_All_Data.csv", encoding='latin-1')

Cleaning the Dataset "Production" 

In [47]:
df2 = df.dropna(how = 'any') # removing all missing data 
df2 = df2[df2["Area Code"]<1000] # removing the summerised Areas (Africa as a hole etc.)
df2 = df2.drop(["Area Code", "Area Code (M49)","Item Code (CPC)" ,"Item Code", "Element Code"], axis=1) # removing not used collums
df2 = df2.drop(df2.columns[df2.columns.str.endswith('F')], axis=1) # removing all collums wich say how the data was collectet. Big loss of information!! 
df2 = pd.melt(df2, id_vars=['Area', "Item", "Element", "Unit"], var_name='Year', value_name='Value') # reshaping the df to create a single year collum 
df2['Year'] = df2['Year'].str[1:] # removin the letter of befor the years
df2 

,Area,Item,Element,Unit,Year,Value
0,Afghanistan,Apples,Area harvested,ha,Y1961,2220.00
1,Afghanistan,Apples,Yield,hg/ha,Y1961,68018.00
2,Afghanistan,Apples,Production,tonnes,Y1961,15100.00
3,Afghanistan,Apricots,Area harvested,ha,Y1961,4820.00
4,Afghanistan,Apricots,Yield,hg/ha,Y1961,66390.00
...,...,...,...,...,...,...
1772777,Zimbabwe,Sugar Crops Primary,Yield,hg/ha,Y2021,734043.00
1772778,Zimbabwe,Sugar Crops Primary,Production,tonnes,Y2021,3450000.00
1772779,Zimbabwe,Vegetables Primary,Area harvested,ha,Y2021,35148.00
1772780,Zimbabwe,Vegetables Primary,Yield,hg/ha,Y2021,66770.00
